In [ ]:
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision
import os, shutil
from sklearn.metrics import *
from tqdm.notebook import tqdm

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, 64, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, 128, shuffle=False, drop_last=False)

In [ ]:
def train_epoch(model, dataloader, loss, optimizer, scheduler, device, n_epoch):
  pbar = tqdm(dataloader, total = len(dataloader))
  loss_p = [0, 0]
  preds, targets = [], []
  for (data, labels) in pbar:
    data = data.to(device)
    labels = labels.to(device)
    outputs = model(data)
    cur_loss = loss(outputs, labels)
    loss_p[0] += cur_loss.item() * labels.shape[0]
    loss_p[1] += labels.shape[0]
    targets += labels.detach().cpu().numpy().tolist()
    preds += outputs.detach().argmax(dim=1).cpu().numpy().tolist()
    cur_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    pbar.set_description(f"Train_loss: {loss_p[0] / loss_p[1]}")
  if scheduler is not None:
    scheduler.step()
  return (loss_p[0] / loss_p[1], accuracy_score(targets, preds))

@torch.no_grad
def eval_epoch(model, dataloader, loss, optimizer, scheduler, device, n_epoch):
  pbar = tqdm(dataloader, total = len(dataloader))
  loss_p = [0, 0]
  model.eval()
  preds, targets = [], []
  for (data, labels) in pbar:
    data = data.to(device)
    labels = labels.to(device)
    outputs = model(data)
    cur_loss = loss(outputs, labels)
    loss_p[0] += cur_loss.item() * labels.shape[0]
    loss_p[1] += labels.shape[0]
    targets += labels.detach().cpu().numpy().tolist()
    preds += outputs.detach().argmax(dim=1).cpu().numpy().tolist()
    pbar.set_description(f"Test_loss: {loss_p[0] / loss_p[1]}")
  return (loss_p[0] / loss_p[1], accuracy_score(targets, preds))

def train_model(model, train_loader, test_loader, loss, optimizer, scheduler=None, device='cpu', n_epochs=10):
  pbar = tqdm(range(n_epochs), total = n_epochs)
  model = model.to(device)
  for epoch in pbar:
    train_loss, train_acc = train_epoch(model, train_loader, loss, optimizer, scheduler, device, epoch)
    test_loss, test_acc = eval_epoch(model, test_loader, loss, optimizer, scheduler, device, epoch)
    pbar.set_description(f"Epoch: {epoch}, Train Loss: {train_loss}, Test_loss: {test_loss}, Test Accuracy: {test_acc}")
  return model

In [ ]:
model = torchvision.models.resnet18()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = None
device = 'cpu'
n_epochs=5
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
model.fc = nn.Linear(512, 10)

In [ ]:
train_model(model, train_loader, test_loader, loss, optimizer, scheduler, device, n_epochs)